# Caffe的Python接口学习(6)：用训练好的模型（caffemodel）来分类新的图片

经过前两篇的学习，我们已经训练好了一个caffemodel模型，并生成了一个deploy.prototxt文件，现在我们就利用这个文件来对一个新的图片进行分类预测。

我们从mnist数据集的test集中随便找一张图片，用来进行实验。

``` python
# coding=utf-8

import caffe
import numpy as np

# 父目录
root = '/root/Repository/caffe/examples/mine/'
# deploy文件
deploy = root + 'mnist/deploy.prototxt' # deploy文件
# 训练好的caffemodel
caffe_model = root + 'mnist/lenet_iter_9380.caffemodel'
# 随机找的一张待测图片
img = root + 'mnist/test/5/00008.png'
# 类别名称文件，将数字标签转换回类别名称
labels_filename = root + 'mnist/test/labels.txt'

# 加载model和network
net = caffe.Net(deploy, caffe_model, caffe.TEST)

# 图片预处理设置
# 设定图片的shape格式(1, 3, 28, 28)
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
# 改变维度的顺序，由原始图片(28, 28, 3)变为(3, 28, 28)
transformer.set_transpose('data', (2, 0, 1))
# 减去均值，前面训练模型时没有减去均值，这里就不用了
# transformer.set_mean('data', np.load(mean_file).mean(1).mean(1))
# 缩放到[0, 255]之间
transformer.set_raw_scale('data', 255)
# 交换通道，将图片由RGB变为BGR
transformer.set_channel_swap('data', (2, 1, 0))

# 加载图片
im = caffe.io.load_image(img)
# 执行上面设置的图片预处理操作，并将图片载入到blob中
net.blobs['data'].data[...] = transformer.preprocess('data', im)

# 执行测试
out = net.forward()

# 读取类别名称文件
labels = np.loadtxt(labels_filename, str, delimiter='\t')
# 取出最后一层(Softmax)属于某个类别的概率值，并打印
prob = net.blobs['Softmax1'].data[0].flatten()
print(prob)
# 按概率值排序，取出最大值所在的序号
order = prob.argsort()[-1]
# 将该序号转换成对应的类别名称，并打印
print('the class is: ', labels[order])
```

最后输出： the class is: 5

分类正确。

如果时预测多张图片，可把上面这个文件写成一个函数，然后进行循环预测就可以了。